<a href="https://colab.research.google.com/github/laura314159265/poly/blob/main/PR%C3%80CTICA_GUIADA/Crank_em_vull_matar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Bueno toca Crank-Nicolson  \
Si fas com derivades numèriques i reordenes queda:
$T_{i+1}^n (1+2\gamma)-\gamma·T_{i+1}^{n+1}-\gamma · T_{i+1}^{n-1} = T_i^n + \gamma · T_i^{n+1} + \gamma ·T_i
^{n-1} + Δt$\
que es pot escriure com un sistema de matrius to jodido pero bueno tenim llibreries\
Tenim dues matrius tridiagonals.\
 La de l'esquerre, matriu A, te per offdiagonal $\gamma$ i per diagonal $(1+2\gamma)$ i multiplica un vector de les T a temps i+1\
 La de la dreta, matriu B, te per offdiagonal $-\gamma$ i per diagonal $(1-2\gamma)$ i multiplica un vector de les T a temps i i li suma $Δt$\
 Necessitem puto invertir la matriu de l'esquerre, multiplicar tot a la dreta per l'inversa i tenim el vector d'incognites aïllat amb coses que coneixem (vamos que tenim el sistema resolt)

In [ ]:
import numpy as np
from scipy.sparse import tridiagonal
from scipy.sparse.linalg import inv

# Ejemplo de una matriz tridiagonal
# Definir las tres diagonales
n = 3
main_diag = [2, 3, 4]
off_diag = [1, 1] # Para la diagonal inferior y superior
diagonals = [off_diag, main_diag, off_diag]

# Crear la matriz tridiagonal dispersa
mat_tridiagonal = tridiagonal(diagonals[0], diagonals[1], diagonals[2])

# Convertir a una matriz densa para usar numpy.linalg.inv (para matrices pequeñas)
mat_dense = mat_tridiagonal.toarray()
mat_inv_np = np.linalg.inv(mat_dense)
print("Inversa usando NumPy:\n", mat_inv_np)

# Usar scipy.sparse.linalg.inv (para un mejor rendimiento con matrices grandes)
mat_inv_scipy = inv(mat_tridiagonal)

# El resultado es una matriz dispersa, podemos ver su representación densa
print("\nInversa usando SciPy:\n", mat_inv_scipy.toarray())